# Морфологические процессоры для русского языка

Существует два основных вида морфологических процессоров: морфологические анализаторы и морфологические словари.
Морфологический анализатор определяет часть речи слова по его парадигме. Умеет определять вероятность того или иного разбора. 
Морфологический словарь определяет часть речи на основе разметки словаря / корпуса. Может возвращать несколько разборов каждого слова, причем для каждого разбора будет определена его частота в размеченном корпусе. 
* mystem – морфологический анализатор Yandex (https://tech.yandex.ru/mystem/) . Утверждается, что 3-я версия mystem снимает омонимию.
    *  pymystem3 –  обертка для Python (https://github.com/Digsolab/pymystem3)
* pymorphy2 – морфологический словарь на основе словарей Открытого корпуса (https://pymorphy2.readthedocs.org/en/latest/)

Морфологические процессоры чаще всего используются:
1. для лемматизации (lemmatization)
2. для определения части речи (Part-of-speech tagging, POS tagging)

Эти задачи, разумеется, связанные, но вызовы морфологических процессоров различаются.

## Лемматизация

In [17]:
from pymystem3 import Mystem
text = u"Красивая мама красиво мыла раму"
text = 'Глокая куздра штеко будланула бокра и курдячит бокренка'
m = Mystem()
lemmas = m.lemmatize(text)
print(''.join(lemmas))

глокай куздра штеко будланул бокра и курдячить бокренок



In [18]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
#возьмем 0 разбор – самый частый в ОК 

for word in text.split():
    print(morph.parse(word)[0].normal_form)

глокать
куздра
штёкий
будланула
бокра
и
курдячать
бокрёнок


In [5]:
for i in morph.parse(u'раму'):
    print(i)

Parse(word='раму', tag=OpencorporaTag('NOUN,inan,masc,Geox sing,datv'), normal_form='рам', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'раму', 32, 2),))
Parse(word='раму', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='рама', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'раму', 55, 3),))


In [19]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
#возьмем 0 разбор – самый частый в ОК 

for word in text.split():
    print(morph.parse(word)[0].normal_form)


# print morph.parse(u'стекло')
# print len(morph.parse(u'стекло'))
for i in morph.parse(u'раму'):
    print(i.normal_form, i.tag.cyr_repr)

глокать
куздра
штёкий
будланула
бокра
и
курдячать
бокрёнок
рам СУЩ,неод,мр,гео ед,дт
рама СУЩ,неод,жр ед,вн


Проверим, как обстоят дела с омонимией. 

In [9]:
text = u"Он видел их семью своими глазами"
lemmas = m.lemmatize(text)
print(''.join(lemmas))
print('***')
for word in text.split():
    print(morph.parse(word)[0].normal_form)


он видеть их семья свой глаз

***
он
видеть
они
семь
свой
глаз


## Определение части речи

In [12]:
import json
text = u"Красивая мама красиво мыла раму"
gram_info = m.analyze(text)
print("full info:", json.dumps(gram_info, ensure_ascii=False))
for i in range(len(gram_info)):
    if not gram_info[i]["text"].isspace():
        print(gram_info[i]["text"])
        print('Gram info:', gram_info[i]["analysis"][0]["gr"])
        print('POS tag:', gram_info[i]["analysis"][0]["gr"].split(',')[0].split('=')[0])
    

full info: [{"analysis": [{"gr": "A=им,ед,полн,жен", "lex": "красивый"}], "text": "Красивая"}, {"text": " "}, {"analysis": [{"gr": "S,жен,од=им,ед", "lex": "мама"}], "text": "мама"}, {"text": " "}, {"analysis": [{"gr": "ADV=", "lex": "красиво"}], "text": "красиво"}, {"text": " "}, {"analysis": [{"gr": "V,несов,пе=прош,ед,изъяв,жен", "lex": "мыть"}], "text": "мыла"}, {"text": " "}, {"analysis": [{"gr": "S,жен,неод=вин,ед", "lex": "рама"}], "text": "раму"}, {"text": "\n"}]
Красивая
Gram info: A=им,ед,полн,жен
POS tag: A
мама
Gram info: S,жен,од=им,ед
POS tag: S
красиво
Gram info: ADV=
POS tag: ADV
мыла
Gram info: V,несов,пе=прош,ед,изъяв,жен
POS tag: V
раму
Gram info: S,жен,неод=вин,ед
POS tag: S


In [14]:
for word in text.split():
    print(word)
    print('Gram info:', morph.parse(word)[0].tag)
    print('POS tag:', morph.parse(word)[0].tag.POS)

Красивая
Gram info: ADJF,Qual femn,sing,nomn
POS tag: ADJF
мама
Gram info: NOUN,anim,femn sing,nomn
POS tag: NOUN
красиво
Gram info: ADVB
POS tag: ADVB
мыла
Gram info: NOUN,inan,neut sing,gent
POS tag: NOUN
раму
Gram info: NOUN,inan,masc,Geox sing,datv
POS tag: NOUN


Синтаксис pymystem3 существенно сложнее, POS тэги не хранятся в явном виде. Для того, что бы узнать POS тэг приходится использовать чудовищную строчку gram_info[i]["analysis"][0]["gr"].split(',')[0].split('=')[0] . 

In [16]:
import json
gram_info = m.analyze(text)
gram_info,gram_info[0]["analysis"][0]["gr"].split(',')[0].split('=')[0]

([{'analysis': [{'gr': 'A=им,ед,полн,жен', 'lex': 'красивый'}],
   'text': 'Красивая'},
  {'text': ' '},
  {'analysis': [{'gr': 'S,жен,од=им,ед', 'lex': 'мама'}], 'text': 'мама'},
  {'text': ' '},
  {'analysis': [{'gr': 'ADV=', 'lex': 'красиво'}], 'text': 'красиво'},
  {'text': ' '},
  {'analysis': [{'gr': 'V,несов,пе=прош,ед,изъяв,жен', 'lex': 'мыть'}],
   'text': 'мыла'},
  {'text': ' '},
  {'analysis': [{'gr': 'S,жен,неод=вин,ед', 'lex': 'рама'}], 'text': 'раму'},
  {'text': '\n'}],
 'A')

In [6]:
verb = u'быть'
verb_parse = morph.parse(verb)[0]
femn_gram = {'VERB','impf','femn','sing','past','indc'}
musc_gram = {'VERB','impf','musc','sing','past','indc'}

femn_name = [u'Катя',u'Оля',u'Юля']
musc_name = [u'Митя',u'Сережа',u'Андрей']


name = input(u'Привет, как тебя зовут? ')
if name in femn_name:
    print(u'%s, где ты %s всю прошлую ночь?' %(name,verb_parse.inflect(femn_gram).word))
elif name in musc_name:
    print(u'%s, где ты %s всю прошлую ночь?' %(name,verb_parse.inflect(musc_gram).word))
else:
    print(u'%s, как прошла ночь?' %(name))

Привет, как тебя зовут? Катя
Катя, где ты была всю прошлую ночь?


In [7]:
import nltk

## Задание 1
1. Записать произвольный текст в файл.
2. Считать текст из файла.
3. Используя оба морфологических процессора, найти все существительные в тексте. На выходе должно быть 2 независимых списка существительных.
4. Посчитать количество совпадений и различий.
5. Используя nltk.FreqDist(), посчитать частоты всех существительных. На выходе должно быть 2 независимых частотных словаря. 
6. Найти 10 самых частых существительных в каждом частотном словаре (nltk.FreqDist().most_common()) и вывести их в цикле.
7. Можно ли эти существительные считать *ключевыми словами*? 

## Задание 2
1. Написать программу, которая ведет осмысленный диалог и склоняет существительные и глаголы. 

## Задание 3
1. Найти два текста разных стилей (или жанров), например, описание продукта из интернет-магазина и инструкцию по его употреблению.
2. Использовать любой морфологический процессор для разбора обоих текстов. На выходе должно быть два разобранных текста: каждый текст представлен списком из списков, каждый мини-список – это пара [слово, POS тег]. 
3. Посчитать, сколько раз какие части речи употребляются в каждом тексте (можно использовать nltk.FreqDist()).
4. Можно ли на основе частот разных частей речи делать выводы о стилевых различиях между текстами?

# Литература

* Manning & Shuetze: Ch. 10 – Part of Speech Tagging

* NLTK Book: Ch. 1, 3.1   Frequency Distributions – частотные словари

* Статья Сегаловича про mystem – http://download.yandex.ru/company/iseg-las-vegas.pdf
